# Cross Sectional Lifetime Dependence

This notebook will attempt to analyze the electron lifetime (roughly, using the "LifetimeEstimation" class) as a function of (x,y) channels of the charge tile in the long TPC at Stanford. The analysis will be carried out using data from Runs 29, 30, 31, and 34, purification test runs with high statistics, on datasets with known lifetimes calculated. (Purification tests are performed by injection Rn-220 gas into the chamber, then waiting for the Bi-212 alpha decays to dominate before taking data.)

The following process will be used:
- Separate the data into channels (readouts of individuals/groups preserved); plot tile energy vs. drift time per channel
- Fit a lifetime to the plots
- Analyze the stastics of all the lifetimes (mainly mean and std. dev)
- Repeat for multiple datasets, multiple runs to see if any patterns exist

Anything else? I can't think of anything right now

In [1]:
# Import Modules
import sys
import cycler
import numpy as np
import matplotlib.pyplot as plt
from StanfordTPCAnalysis.StruckAnalysisConfiguration import StruckAnalysisConfiguration
from StanfordTPCAnalysis.WaveformAnalysis import Waveform as wv
from StanfordTPCAnalysis.ParseStruck import NGMBinaryFile
import time
import pickle
from StanfordTPCAnalysis.LifetimeEstimation import LifetimeEstimation as LE

# Augment the path environment variable
sys.path.insert(0,'../..')

# Set up Plot Stuff
plt.rcParams['axes.prop_cycle'] = cycler.cycler(color='bgrmyk')
prop_cycle = plt.rcParams['axes.prop_cycle']
colors = prop_cycle.by_key()['color']

plt.rcParams['figure.figsize'] = [10, 8]
plt.rcParams['font.size'] = 12

SyntaxError: invalid syntax (LifetimeEstimation.py, line 151)

Import the datasets, files and stuff

In [ ]:
# Run 33, 34 Info (My permission to other sets is potentially denied) 
path_to_reduced_data = '/p/lustre2/nexouser/hardy27/StanfordTPCData/purification_data/'
run_num = 'Run34' # 'Run34'
this_dataset = 'DS04' # 'DS04' 'DS05' 'DS07' 'DS08' 
reduced_num = '/reduced_v9/' # '/reduced_v4/'
# fname = path_to_reduced_data + run_num + '/' + this_dataset + reduced_num + 'reduced_added.p'  
print(run_num)
print(this_dataset)
#all the run configuration paramenters are stored in these three files
config_path = '/DS05/config/Run34_v2/' # '/33rd/' # '/config/Run34_v2/'
# run_parameters_file = path_to_reduced_data + run_num + '/' + this_dataset + config_path + 'Run_Parameters.csv'
# calibrations_file = path_to_reduced_data + run_num + '/' + this_dataset + config_path + 'Calibrations_Xe.csv'
# channel_map_file = path_to_reduced_data + run_num + '/' + this_dataset + config_path + 'Channel_Map.csv'

In [ ]:
# #analysis_config object loads all these paramenters
# analysis_config = StruckAnalysisConfiguration.StruckAnalysisConfiguration()

# analysis_config.GetRunParametersFromFile( run_parameters_file, sheet = this_dataset )
# analysis_config.GetChannelMapFromFile( channel_map_file, sheet = this_dataset )

# with open(fname,'rb') as input_file:
#     data_df = pickle.load(input_file)

In [ ]:
# Define a function that reads all datasets so that I can get the visual cuts on the data
# I'll store the visual cuts for each just to run the data automatically per dataset later in a 
# second function

def OpenFilesForVisCuts(path_to_reduced_data, run_num, this_dataset, reduced_num, config_path):
    # Establish File Paths
    fname = path_to_reduced_data + run_num + '/' + this_dataset + reduced_num + 'reduced_added.p' 
    run_parameters_file = path_to_reduced_data + run_num + config_path + 'Run_Parameters.csv'
    calibrations_file = path_to_reduced_data + run_num + config_path + 'Calibrations_Xe.csv'
    channel_map_file = path_to_reduced_data + run_num + config_path + 'Channel_Map.csv'
    # Populate Analysis_Config object
    analysis_config = StruckAnalysisConfiguration.StruckAnalysisConfiguration()
    analysis_config.GetRunParametersFromFile( run_parameters_file, sheet = this_dataset )
    analysis_config.GetChannelMapFromFile( channel_map_file, sheet = this_dataset )
    # Read the Data
    with open(fname,'rb') as input_file:
        data_df = pickle.load(input_file)
    # Extract Tile Data, Drift Time, Plot for Cuts
    charge_energy = data_df['TotalTileEnergy']
    sipm_energy = data_df['TotalSiPMEnergy']
    time_of_max_channel = data_df['TimeOfMaxChannel']
    drift_time = (time_of_max_channel - analysis_config.run_parameters['Pretrigger Length [samples]']) \
               * analysis_config.run_parameters['Sampling Period [ns]'] / 1000.

    # Plot uncut Tile Energy, SiPM Energy vs. Drift Time
    plt.figure(0)
    plt.plot(drift_time,charge_energy,'o',color=(0.,0.,1.,0.5),markersize=5.,\
         markeredgecolor=(0.,0.,0.,0.))
    plt.xlabel("Drift time (µs)")
    plt.ylabel("Charge Tile Energy (ADC Counts)")
    plt.title("Tile Energy vs. Drift Time, " + run_num + ", " + this_dataset)
    return data_df, analysis_config, drift_time, charge_energy, sipm_energy


data_df, analysis_config, drift_time, charge_energy, sipm_energy = OpenFilesForVisCuts(path_to_reduced_data, run_num, this_dataset, reduced_num, config_path)

In [ ]:
print(f'Size of Charge Energy: {np.shape(charge_energy)}')
# Add some cuts
sipm_lower_bound = 25000
sipm_upper_bound = 1000000
charge_lower_bound = 50 # ADC Counts 
charge_upper_bound = 700  # ADC Counts 

# Apply Cuts to Data to Visualize the Alpha Band; from this, guess the lifetime and intercept
mask = (data_df['TotalTileEnergy']>charge_lower_bound) & (data_df['TotalTileEnergy']<charge_upper_bound) \
        & (data_df['TotalSiPMEnergy']>sipm_lower_bound) & (data_df['TotalSiPMEnergy']<sipm_upper_bound) \
        & (data_df['NumTileChannelsHit'] < 3) & (data_df['IsFull3D'])

plt.figure(0)
plt.plot(drift_time[mask],charge_energy[mask],'o',color=(0.,0.,1.,0.5),markersize=5.,\
         markeredgecolor=(0.,0.,0.,0.))
plt.xlabel("Drift time (µs)")
plt.ylabel("Charge Tile Energy (ADC Counts)")
plt.title("Tile Energy vs. Drift Time, " + run_num + ", " + this_dataset)

In [ ]:
# Try a fit for the lifetime on all the data if the lifetime isn't known
start_drift = 10 # us 
end_drift = 45 # us
intercept_guess = 600 # ADC Counts
lifetime_guess = 80 # us 

lifetime, sigma_lifetime, intercept, sigma_intercept, r_squared,\
    drift_slices, centroids, centroids_stds = LE.binnedElectronLifetimeFit(intercept_guess,\
                                                                           lifetime_guess,start_drift,end_drift,\
                                                                           data_df, analysis_config,sipm_lower_bound,\
                                                                           sipm_upper_bound,charge_lower_bound,\
                                                                           charge_upper_bound, run_num, this_dataset,\
                                                                           plotFlag=False,verbose=False)
total_lifetime = lifetime
total_std_lifetime = sigma_lifetime
mean_x = np.mean(drift_slices)
mean_y = np.mean(centroids)
plt.figure(95)
plt.plot(drift_time[mask],charge_energy[mask],'o',color=(0.,0.,1.,0.5),markersize=5.,\
         markeredgecolor=(0.,0.,0.,0.),label='Data')
plt.errorbar(drift_slices,centroids,yerr=centroids_stds,fmt='o',color='black',markersize=5.,\
         markeredgecolor=(0.,0.,0.,0.),label='Alpha Band Fit Centroids (Bins are Left Edges)')
plt.plot(drift_slices,intercept*np.exp(-drift_slices/lifetime),'-',\
         color=(0.,0.,0.,1),linewidth=1.5,markeredgecolor=(0.,0.,0.,0.),\
         label=f'Fit: y = (${intercept:.2f}  \u00B1  ${sigma_intercept:.2f})*exp(-x/(${lifetime:.2f}  \u00B1  ${sigma_lifetime:.2f}))')
plt.plot(mean_x,mean_y,'o',color=(0.,0.,0.,0.01),markersize=0.01,markeredgecolor=(0.,0.,0.,0.),\
         label=f'Electron Lifetime: ${lifetime:.2f}  \u00B1  ${sigma_lifetime:.2f} µs')
plt.plot(mean_x,mean_y,'o',color=(0.,0.,0.,0.01),markersize=0.01,markeredgecolor=(0.,0.,0.,0.),\
         label=f'R^2 of Log Fit: {r_squared:.4f}')
plt.xlim([0,np.amax(drift_time[mask])])
plt.ylim([0,1.05*np.amax(charge_energy[mask])])
plt.xlabel("Drift time (µs)")
plt.ylabel("Charge Tile Energy (ADC Counts)")
plt.title("Electron Lifetime, " + run_num + ", "+this_dataset+" ("+str(np.size(centroids))+" Fit Points)")
plt.legend(bbox_to_anchor=(1.1,1.0))

Break up the data into channels (Tile Strip Charge Energies)

In [ ]:
for colname in data_df.columns:
    print(colname)

In [ ]:
channel_name = 'TileStrip Y7/8 Charge Energy'
foo = data_df[channel_name]
print(f'Charge Lower Bound Cause Im stupid: {charge_lower_bound}')
print(f'SiPM Lower Bound Cause Im stupid: {sipm_lower_bound}')
print(f'SiPM Upper Bound Cause Im stupid: {sipm_upper_bound}')
numXChannels = 15
numYChannels = 15
sipm_lower_bound = 5000
sipm_upper_bound = 75000
mask_individual = (data_df[channel_name]>0) &\
                    (data_df[channel_name]<100) \
        & (data_df['TotalSiPMEnergy']>sipm_lower_bound) & (data_df['TotalSiPMEnergy']<sipm_upper_bound) \
        & (data_df['NumTileChannelsHit'] < 3) & (data_df['IsFull3D'])\
        & (data_df['TimeOfMaxChannel']> 0)

print(f'Number of Events in Channel X19, Dumb Mask: {np.shape(foo[mask_individual])}')
print(f'Number of Events in Channel X19, Sensible Mask: {np.shape(foo[mask])}')

print(f'Thats what the point of the mask is: {np.where(mask_individual==1)} ')

# plt.figure(0)
# plt.plot(drift_time[mask],foo[mask],'o',color=(0.,0.,1.,0.5),markersize=5.,\
#          markeredgecolor=(0.,0.,0.,0.),label='Data')
# plt.title('Tile Energy vs Drift Time, X19, Normal Mask')
# plt.figure(1)
# plt.hist(foo[mask],50)
# plt.title('Hist, Normal Mask')

plt.figure(2)
plt.plot(drift_time[mask_individual],foo[mask_individual],'o',color=(0.,0.,1.,0.5),markersize=5.,\
         markeredgecolor=(0.,0.,0.,0.),label='Data')
plt.title('Tile Energy vs Drift Time, X20')
plt.xlabel('Drift Time (us)')
plt.ylabel('Charge Energy (ADC Counts)')
# plt.figure(3)
# plt.hist(foo[mask_individual],50)
# plt.title('Hist, Dumb Mask')

# plt.figure(4)
# plt.hist(bar,100,range=[0,100])
# plt.title('Hist of Drift Time, X19 Only')

# plt.figure(5)
# plt.plot(drift_time,foo,'o',color=(0.,0.,1.,0.5),markersize=5.,\
#          markeredgecolor=(0.,0.,0.,0.),label='Data')
# plt.title('Tile Energy vs Drift Time, No Mask')



print('')

In [ ]:
# I'm going to try using the Dumb Mask, fit for a lifetime between 10 and 45 microseconds
intercept_guess_individual = 90 # ADC Counts 
charge_lower_bound = 0
charge_upper_bound = 100
channel_name = 'TileStrip Y7/8 Charge Energy'
charge_energy_plot = data_df[channel_name]

lifetime, sigma_lifetime, intercept, sigma_intercept, r_squared,\
    drift_slices, centroids, centroids_stds = LE.binnedElectronLifetimeFitChannel(intercept_guess_individual,\
                                                                           lifetime_guess,start_drift,end_drift,\
                                                                           data_df, channel_name, analysis_config,\
                                                                            sipm_lower_bound,\
                                                                           sipm_upper_bound,charge_lower_bound,\
                                                                           charge_upper_bound, run_num, this_dataset,\
                                                                           plotFlag=True,verbose=True)
mean_x = np.mean(drift_slices)
mean_y = np.mean(centroids)
plt.figure(95)
plt.plot(drift_time[mask_individual],charge_energy_plot[mask_individual],'o',color=(0.,0.,1.,0.5),markersize=5.,\
         markeredgecolor=(0.,0.,0.,0.),label='Data')
plt.errorbar(drift_slices,centroids,yerr=centroids_stds,fmt='o',color='black',markersize=5.,\
         markeredgecolor=(0.,0.,0.,0.),label='Alpha Band Fit Centroids (Bins are Left Edges)')
plt.plot(drift_slices,intercept*np.exp(-drift_slices/lifetime),'-',\
         color=(0.,0.,0.,1),linewidth=1.5,markeredgecolor=(0.,0.,0.,0.),\
         label=f'Fit: y = (${intercept:.2f}  \u00B1  ${sigma_intercept:.2f})*exp(-x/(${lifetime:.2f}  \u00B1  ${sigma_lifetime:.2f}))')
plt.plot(mean_x,mean_y,'o',color=(0.,0.,0.,0.01),markersize=0.01,markeredgecolor=(0.,0.,0.,0.),\
         label=f'Electron Lifetime: ${lifetime:.2f}  \u00B1  ${sigma_lifetime:.2f} µs')
plt.plot(mean_x,mean_y,'o',color=(0.,0.,0.,0.01),markersize=0.01,markeredgecolor=(0.,0.,0.,0.),\
         label=f'R^2 of Log Fit: {r_squared:.4f}')
plt.xlim([0,np.amax(drift_time[mask_individual])])
plt.ylim([0,1.05*np.amax(charge_energy_plot[mask_individual])])
plt.xlabel("Drift time (µs)")
plt.ylabel("Charge Tile Energy (ADC Counts)")
plt.title("Electron Lifetime, " + run_num + ", "+this_dataset+", "+channel_name+" ("+str(np.size(centroids))+" Fit Points)")
plt.legend(bbox_to_anchor=(1.1,1.0))

After testing this notebook out with a few channels in a dataset that works with the class, I've encountered some really daunting facts about carrying out this task:
- Each run has its own naming conventions for the run parameters and channel map files, so I'll have to change the names to load datasets every time
- Each channel in each dataset has its own name with its own naming convention, so there's no great way that I know of to extract all the channel names, unless there's something I can do with a grep if it can extract the whole title after grepping only part of it, and then store the names in some long array
- Each channel (or group of channels) has its own proprietary cuts and intercept guess, so if I were to run this program for a whole list of datasets, I would have to cut each channel's data appropriately and make accurate intercept guesses, then record and store these values in long arrays
- If I wanted to estimate the lifetime for a group of channels, I would have to modify existing functions to change the masking, like I did for the individual channels (my class/code needs a serious cleaning)

NOTE after the meeting:
- Don't be stupid; if the channels have such low counts, then it's likely all noise there. I'll have to group together tiles accordingly in a meaningful way to get meaningful results out of this.
- It seems like I should explore the plotting of charge energy vs total drift time and individual drift time to see how different they are (plotted with the same cuts, of course)

In [ ]:
# This section will contain the massive arrays that contain all the information about the cuts and intercept guesses
# for a single dataset
run34_ds04_strip_names = np.array(['X1/2','Y25/26','Y29/30','Y1/2','Y7/8','X7/8','X21/22','Y5','Y3/4','Y19/20',\
                                   'Y17/18','Y21/22','Y11/12','X11/12','X29/30','X25/26','X17/18','X15/16',\
                                  'X27/28','X5/6','Y15/16','Y23/24','X3/4','Y9/10','Y13/14','X13/14','X20',\
                                  'X23','X9/10','Y27/28'],dtype=str)
run34_ds04_intercept_guesses = np.array([78,42,93,92,90,93,90,68,95,87,87,90,90,87,93,93,93,90,90,105,100,\
                                         110,100,110,115,115,80,85,100,100],dtype=float)
run34_ds04_charge_energy_lower_bound = 0
run34_ds04_charge_energy_upper_bounds = np.array([100,50,100,100,100,100,100,75,110,100,100,100,100,100,100,\
                                                  100,100,100,100,150,120,120,120,110,130,130,90,90,120,120],dtype=int)
run34_ds04_sipm_energy_lower_bounds = 1000*np.array([5,5,5,5,5,2,2,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,\
                                                    5],dtype=int)
run34_ds04_sipm_energy_upper_bounds = 1000*np.array([75,75,75,81,75,70,90,70,100,80,80,80,71,71,95,80,75,80,80,\
                                                     80,80,80,80,80,75,80,75,75,75,75],dtype=int)
channel_vis = 'Y7/8'

In [ ]:
def overlap(start1, end1, start2, end2):
    return (start1 <= start2 <= end1 or
        start1 <= end2 <= end1 or
        start2 <= start1 <= end2 or
        start2 <= end1 <= end2)

# Define a function that calculates the lifetimes of all the strips, omits outliers from the Gaussian fitting,
# if necessary, and refits, then stores all the data
def AllChannelLifetimeFitsDataset(calculated_lifetime, calculated_sigma_lifetime, data_df,lifetime_guess,intercept_guesses,start_drift,end_drift, channel_names, analysis_config,sipm_lower_bounds,sipm_upper_bounds,charge_lower_bound, charge_upper_bounds, run_num, this_dataset,channel_vis, plotFlag=False,verbose=False):
    lifetimes_array = np.zeros((np.shape(intercept_guesses)),dtype=float)
    std_lifetimes_array = np.zeros((np.shape(intercept_guesses)),dtype=float)
    variances_of_lifetimes = np.zeros((np.shape(intercept_guesses)),dtype=float)
    overlap_bools = np.zeros((np.shape(intercept_guesses)),dtype=bool)
    for ndx in range(len(intercept_guesses)):
        if run34_ds04_strip_names[ndx] == channel_vis:
             plotFlag = True
        else:
             plotFlag = False
        channel_name = 'TileStrip '+channel_names[ndx]+' Charge Energy'
        lifetime, sigma_lifetime, intercept, sigma_intercept, r_squared,\
                 drift_slices, centroids, centroids_stds = LE.binnedElectronLifetimeFitChannel(intercept_guesses[ndx],\
                                                                           calculated_lifetime,start_drift,end_drift,\
                                                                           data_df, channel_name, analysis_config,\
                                                                            sipm_lower_bounds[ndx],\
                                                                           sipm_upper_bounds[ndx],charge_lower_bound,\
                                                                           charge_upper_bounds[ndx], run_num, this_dataset,\
                                                                           plotFlag,verbose)
        # Plot the Data with Centroids and Lietime Fit Overlain
        if run34_ds04_strip_names[ndx] == channel_vis:
            mean_x = np.mean(drift_slices)
            mean_y = np.mean(centroids)
            plt.figure(95)
            plt.plot(drift_time[mask_individual],charge_energy_plot[mask_individual],'o',color=(0.,0.,1.,0.5),markersize=5.,\
                 markeredgecolor=(0.,0.,0.,0.),label='Data')
            plt.errorbar(drift_slices,centroids,yerr=centroids_stds,fmt='o',color='black',markersize=5.,\
                 markeredgecolor=(0.,0.,0.,0.),label='Alpha Band Fit Centroids (Bins are Left Edges)')
            plt.plot(drift_slices,intercept*np.exp(-drift_slices/lifetime),'-',\
                 color=(0.,0.,0.,1),linewidth=1.5,markeredgecolor=(0.,0.,0.,0.),\
                 label=f'Original Fit: y = (${intercept:.2f}  \u00B1  ${sigma_intercept:.2f})*exp(-x/(${lifetime:.2f}  \u00B1  ${sigma_lifetime:.2f}))')
            plt.plot(mean_x,mean_y,'o',color=(0.,0.,0.,0.01),markersize=0.01,markeredgecolor=(0.,0.,0.,0.),\
                 label=f'Electron Lifetime: ${lifetime:.2f}  \u00B1  ${sigma_lifetime:.2f} µs')
            plt.plot(mean_x,mean_y,'o',color=(0.,0.,0.,0.01),markersize=0.01,markeredgecolor=(0.,0.,0.,0.),\
                 label=f'R^2 of Log Fit: {r_squared:.4f}')
            plt.xlim([0,np.amax(drift_time[mask_individual])])
            plt.ylim([0,1.05*np.amax(charge_energy_plot[mask_individual])])
            plt.xlabel("Drift time (µs)")
            plt.ylabel("Charge Tile Energy (ADC Counts)")
            plt.title("Electron Lifetime, " + run_num + ", "+this_dataset+", "+channel_name+" ("+str(np.size(centroids))+" Fit Points)")
            plt.legend(bbox_to_anchor=(1.1,1.0))
        
        lifetimes_array[ndx] = lifetime
        std_lifetimes_array[ndx] = sigma_lifetime
        overlap_bools[ndx] = overlap(lifetime-sigma_lifetime,lifetime+sigma_lifetime,\
                                    calculated_lifetime-total_std_lifetime, calculated_lifetime+total_std_lifetime)

        variances_of_lifetimes[ndx] = (lifetime-total_lifetime)**2
        
    
    mean_lifetime = np.mean(lifetimes_array)
    std_lifetime = np.std(std_lifetimes_array)
    mean_var_lifetimes = np.mean(variances_of_lifetimes)
    std_var_lifetimes = np.std(variances_of_lifetimes)

    
    return mean_lifetime,std_lifetime,mean_var_lifetimes,std_var_lifetimes,lifetimes_array,std_lifetimes_array,variances_of_lifetimes,overlap_bools

mean_lifetime,std_lifetime,mean_var_lifetimes,std_var_lifetimes,lifetimes_array,std_lifetimes_array,variances_of_lifetimes,overlap_bools = AllChannelLifetimeFitsDataset(total_lifetime,total_std_lifetime, data_df,lifetime_guess,\
                                                                                               run34_ds04_intercept_guesses,start_drift,end_drift,\
                                                                                               run34_ds04_strip_names, analysis_config,\
                                                                                               run34_ds04_sipm_energy_lower_bounds,run34_ds04_sipm_energy_upper_bounds,\
                                                                                               run34_ds04_charge_energy_lower_bound, run34_ds04_charge_energy_upper_bounds,\
                                                                                               run_num, this_dataset,channel_vis, plotFlag=False,verbose=False)
print('Lifetime Per Channel, '+run_num+' '+this_dataset+f': {mean_lifetime} \u00B1 {std_lifetime} µs')
print(f'Lifetime of the Whole Cylinder, '+run_num+' '+this_dataset+f': {total_lifetime} \u00B1 {total_std_lifetime} µs')
print(f'Variances of Channel Lifetimes from "Total" Lifetime per Channel: {mean_var_lifetimes} \u00B1 {std_var_lifetimes} µs')
print(f'All Lifetimes: {lifetimes_array}')
print(f'All Std Lifetimes {std_lifetimes_array}')
print(f'All Variances of Lifetimes from Total Lifetime: {variances_of_lifetimes}')
print(f'Did the Range of the Lifetimes +/- unc. overlap the lifetime +/- unc. of the whole TPC? {overlap_bools}')
print(f'Fraction of Channels with lifetime nearly that of the total lifetime: {np.size(np.where(overlap_bools==True))}/{np.size(overlap_bools)}')

In [ ]:
# Trying to plot the waveforms of the data

# event_indices = foo.index
# print(f'Event Indices: {event_indices}')
# to_plot_df = data_df.iloc[event_indices].sort_values('File') 


# binary_path = path_to_reduced_data + run_num + '/' + this_dataset + "/raw_data/"
# nevents = 10
# evcnt = 0
# filechanged = True
# oldfile = None
# bin_file = None
# for i, row in to_plot_df.iterrows():
#     if(evcnt > nevents):
#         break #just for demonstration purposes, plot only a few
        
#     if(oldfile != row['File']):
#         filechanged = True
        
#     if(filechanged):
#         #change the oldfile 
#         oldfile = row['File']
#         print(f'This is what oldfile puts out: {oldfile}')
#         #load new binary file
#         bin_file = NGMBinaryFile.NGMBinaryFile(binary_path + oldfile, config=analysis_config)
#         filechanged = False
    
#     #load the event number
#     event = Waveform.Event(bin_file, ev_idx, analysis_config)
#     event.plot_event()
#     #smoothing_windows_us = 0.5
#     #the waveform can be smoothed (just for visualization)
#     #event.smooth(smoothing_windows_us)
#     plt.show()
#     evcnt += 1
    

In [ ]:
def overlap(start1, end1, start2, end2):
    """Does the range (start1, end1) overlap with (start2, end2)?"""
    return (
        start1 <= start2 <= end1 or
        start1 <= end2 <= end1 or
        start2 <= start1 <= end2 or
        start2 <= end1 <= end2
    )

start1 = lifetime-sigma_lifetime
end1 = lifetime+sigma_lifetime
start2 = total_lifetime - total_std_lifetime
end2 = total_lifetime + total_std_lifetime

thing = overlap(start1, end1, start2, end2)
print(thing)

In [ ]:
x = np.array([3, 1, 5],dtype=float)[np.newaxis]
print(x)
print(x.transpose())
print(np.matmul(x.transpose(),x))
print(np.linalg.inv(np.matmul(x.transpose(),x)))
y = np.array([2,2.7,2.9],dtype=float)[np.newaxis]
print(y)
print(np.matmul(x.transpose(),y))
beta_ols = np.matmul(np.linalg.inv(np.matmul(x.transpose(),x)),np.matmul(x.transpose(),y))